In [1]:
%%capture
!pip install rioxarray xarray-spatial planetary_computer

In [2]:
import pystac_client
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rioxarray
import shapely
import stackstac
import planetary_computer
from osgeo import gdal
gdal.UseExceptions()

In [3]:
# Define bounds
with rasterio.open("../data_working/damage_rasters/2020.tif") as src:
    bounds_3857 = src.bounds

bounds_4326 = rasterio.warp.transform_bounds(3857, 4326, *bounds_3857)

box = gpd.GeoSeries([shapely.geometry.box(*bounds_3857)], crs=3857)
box.explore()

In [4]:
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

prefire = stac.search(
    bbox=bounds_4326,
    datetime="2020-07-01/2020-07-31",
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 50}},
).item_collection()

postfire = stac.search(
    bbox=bounds_4326,
    datetime="2021-07-01/2021-07-31",
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 50}}
).item_collection()

print(f"Prefire: {len(prefire)} items")
print(f"Postfire: {len(postfire)} items")

Prefire: 10 items
Postfire: 9 items


In [5]:
prefire_stack = (
    stackstac.stack(
        prefire,
        assets=["nir08", "red", "swir22"],  # need this for NDVI and rdNBR
        chunksize=4096,
        bounds=bounds_3857,
        epsg=3857
    )
    .where(lambda x: x > 0, other=np.nan)  # 0 is nodata
)

postfire_stack = (
    stackstac.stack(
        postfire,
        assets=["nir08", "red", "swir22"],  # need this for NDVI and rdNBR
        chunksize=4096,
        bounds=bounds_3857,
        epsg=3857
    )
    .where(lambda x: x > 0, other=np.nan)  # 0 is nodata
)

In [6]:
# Bring both into memory (you should see a 1.2 GB jump in RAM usage)
prefire_stack = prefire_stack.persist()
postfire_stack = postfire_stack.persist()

In [7]:
prefire_median  = prefire_stack.median(dim="time").compute()
postfire_median = postfire_stack.median(dim="time").compute()

In [8]:
def nbr(img):
    nir  = img.sel(band="nir08")
    swir = img.sel(band="swir22")

    return ((nir - swir) / (nir + swir)).clip(-1, 1)

prefire_nbr = nbr(prefire_median).rio.write_crs(3857)
postfire_nbr = nbr(postfire_median).rio.write_crs(3857)

# These are a fraction of a pixel off
postfire_nbr = postfire_nbr.rio.reproject_match(prefire_nbr, resampling=rasterio.enums.Resampling.bilinear)

dnbr = prefire_nbr - postfire_nbr

rdnbr = dnbr / np.sqrt(np.abs(prefire_nbr))
rdnbr = rdnbr.where(np.isfinite(rdnbr)).clip(-2, 2)

In [9]:
# Reproject to match the template
with rioxarray.open_rasterio("../data_working/damage_rasters/2020.tif") as src:
    rdnbr_proj = rdnbr.rio.reproject_match(src, resampling=rasterio.enums.Resampling.average)
    rdnbr_proj.rio.to_raster("../data_working/creek_rdnbr.tif")